# Entropic Risk-Averse GMM: Numerical Experiments 
The codes and the results on this page are based on the paper [[Can and Gurbuzbalaban, 2022]](https://arxiv.org/abs/2204.11292) on entropic risk-averse generalized momentum methods for strongly convex smooth optimization. We consider two types of cost functions: strongly convex smooth quadratic objectives and logistic regression with regularization. We compare the performance of risk-averse GMM and AGD algorithms with the standard GD and AGD algorithms where parameters are chosen as $(\alpha,\beta,\gamma)=(1/L, 0,0)$ and $(\alpha,\beta,\gamma)=(1/L, \frac{1-\sqrt{\mu/L}}{1+\sqrt{\mu/L}},\frac{1-\sqrt{\mu/L}}{1+\sqrt{\mu/L}})$,respectively.

__Packages and setup:__ We require EntropicFOMs package to run the experiments.

    sFOMs.py: Contains the class that generates both quadratic optimization problem (optim_problem="quadratic") and logistic regression on syntethic data (optim_problem="synthetic_logit").
    
    ObjFunctions.py: Stores the quadratic objective function and logistic cost functions.
    
    entropic_ra_sFoms.py: Defines the class for entropic risk-averse algorithms which uses grid search to tune the parameters of the algorithm by systematically trading off convergence rate with EV@R of the algorithm based on the scheme provided in [[Can and Gurbuzbalaban, 2022]](https://arxiv.org/abs/2204.11292).
    
    SynthethicData.py: Generates the syntethic data to be used on logistic regression. 

## 1. Strongly convex smooth quadratic objectives

We first consider the following quadratic optimization problem where the objective function is chosen as 
$$
f(x)=\frac{1}{2}x^\top Q x+ b^\top x+ \frac{\lambda_{\tiny\mbox{reg}}}{2} \Vert x\Vert^2,
$$
in dimension $d=10$ with $b=\frac{1}{\Vert \tilde{b}\Vert}\tilde{b}$ for $\tilde{b}=[1,...,1]\in\mathbb{R}^{10}$, $Q$ is a diagonal matrix with diagonals $Q_{ii}=i^2$ for $i\in\{1,..,10\}$, and $\lambda_{\tiny\mbox{reg}}=5$ is a regularization parameter.

We import the necessary packages, 

In [40]:
from EntropicFOMs import *
from EntropicFOMs.entropic_ra_sFoms import *
import numpy as np
import pandas as pd
import plotly.graph_objects as go 

Notice that the minimum $x_*$ satisfies, 
$$
(Q+\lambda_{\tiny\mbox{reg}}I)x_*= -b.
$$
Hence, we can compute the $x_*$ and $f(x_*)$ to compute the suboptimality and the risk measure for the algorithms:

In [41]:
regularizer =  5
# Define the diagonal matrix Q
eigs=[]
for i in range(10):
  eigs+=[(i+1)**2]
Q=np.diag(eigs)
b=np.ones((len(eigs),1))
b/=np.linalg.norm(b)
# Compute the optimum
x_opt= -np.matmul(np.linalg.inv(Q+regularizer*np.eye(len(eigs))),b)
f_opt=0.5*np.matmul(x_opt.T,np.matmul(Q,x_opt))+np.matmul(b.T,x_opt)+0.5*regularizer*np.matmul(x_opt.T,x_opt)
# Compute the function value at optimum
f_opt=f_opt[0][0]

We define the entropic risk-averse classes for strongly convex quadratic objectives. We assume that the noise on the gradient estimates $\tilde{\nabla} f(x_k)$ is independent Gaussian noise, $\mathcal{N}(0,\sigma^2)$, that is 
$$
\tilde{\nabla} f(x_k)= \nabla f(x_k) + \mathcal{N}(0_{d\times 1},\sigma^2 I_d),
$$
where $0_{d\times 1}$ is the zero vector in $\mathbb{R}^{d}$ and $I_d$ is the identity matrix. We set $\sigma=1$. Then, we created the entropic risk averse classes for both deterministic method, det_gd_quad, and also the noisy first order method, sfoms_quad. Here, we just did a sanity check and observe the convergence of the deterministic algorithm. in the following figure.

In [39]:
# Set the standard deviation of the noise 
noise_st_dev=1

# Define the models 
det_gd_quad=entropic_ra_sFoms(optim_problem="quadratic",
                              eig_vals=np.array(eigs),
                              init_x0="fixed", 
                              noise_std=0,
                              reg_param=regularizer)
# SFOM on quadratic problem
sfoms_quad=entropic_ra_sFoms(optim_problem="quadratic",
                              eig_vals=np.array(eigs),
                              init_x0="fixed", 
                              noise_std=noise_st_dev,
                              reg_param=regularizer)


# Train deterministic GD to obtain the solution
det_gd_quad.train(learning_rate=1/(max(eigs)+det_gd_quad.reg_param)
                  ,stop_criteria=None
                  ,Samples=1,
                  seed=1)
#@title The plot of $f(x_k)$ generated by GD
import plotly.graph_objects as go 
det_path=det_gd_quad.suboptimality_data.values.T[0]
det_quad_fig=go.Figure()
det_quad_fig.add_trace(go.Scatter(y=np.log(det_path)))
det_quad_fig.update_layout(title=r"$\text{The values } f(x_k) \text{,where } \{x_k\}\text{'s are generated by standard GD}$",
                           yaxis_title=r"$\log(f(x_k))$",
                           width=600, height=600)
det_quad_fig.show()

INFO:root:Training model...
learning rate |  0.010 | Iteration momentum |  0.000 | Gradient Momentum |  0.000
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: -0.0156| Norm of gradient:  0.3497
INFO:root:Iteration: 100| Function Value: -0.0254| Norm of gradient:  0.0176
INFO:root:Iteration: 150| Function Value: -0.0254| Norm of gradient:  0.0009
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/quadratic_subopt_GD_Var_0.csv
<ipython-input-39-95cfc58dc7cb>:24: RuntimeWarning:

invalid value encountered in log



Next, we find the risk averse parameters of noisy GD, AGD, and GMM algorithms using grid search on the problem, 
$$  (\alpha_{q},\beta_{q},\gamma_{q}):=\underset{(\alpha,\beta,\gamma)\in\mathcal{S}_q}{\text{argmin}}\quad \bar{E}^q_{1-\zeta}(\alpha,\beta,\gamma)\\
\quad \quad\quad\quad \text{s.t.}\; \frac{\rho^2(\alpha,\beta,\gamma)}{\rho_{q,*}^2} \leq (1+\varepsilon) ,
$$
for given trade-off parameter $\varepsilon$ and  confidence level $\zeta$.

In [42]:
#Compute the risk averse parameters of GMM, AGD, and GD
trade_off=0.25 
conf_level= 0.95 

# Risk-averse AGD parameters
ra_agd_a, ra_agd_b, ra_agd_g = sfoms_quad.raAGD_quad_params(trade_off,conf_level)
print("RA-AGD parameters: \n",
      np.round(ra_agd_a,4), np.round(ra_agd_b,4), np.round(ra_agd_g,4))
# Risk-averse GMM parameters 
ra_gmm_a, ra_gmm_b, ra_gmm_g = sfoms_quad.raTMM_quad_params(trade_off,conf_level)
print("RA-GMM parameters: \n",
      np.round(ra_gmm_a,4), np.round(ra_gmm_b,4), np.round(ra_gmm_g,4))

RA-AGD parameters: 
 0.0004 0.9592 0.9592
RA-GMM parameters: 
 0.0004 0.9592 1.0


Train models using risk risk averse SFOMs and standard first order methods for comparison. 

In [6]:
# Set the smoothness and strong convexity parameters
L=max(sfoms_quad.eig_vals)+sfoms_quad.reg_param
mu= min(sfoms_quad.eig_vals)+sfoms_quad.reg_param

# Number of iterations
max_iteration=300 

# Number of samples
sample_size=50

# Train standard GD
print("Train standard GD") 
stgd_a=1/L
sfoms_quad.train(learning_rate=stgd_a,
                 Samples=sample_size,
                 stop_criteria=max_iteration, 
                 seed=1)
stgd_data=sfoms_quad.suboptimality_data-f_opt

# Retrieve the mean and std of paths
stgd_path_mean= stgd_data.mean(1)
stgd_path_st= stgd_data.std(1)

# Train standard AGD
print("Standard AGD")
stagd_a=1/L
stagd_b= (np.sqrt(L)-np.sqrt(mu))/(np.sqrt(L)+np.sqrt(mu))
sfoms_quad.train(learning_rate=stagd_a,
                 iter_momentum=stagd_b,
                 grad_momentum=stagd_b,
                 Samples=sample_size,
                 stop_criteria=max_iteration,
                 seed=1)
stagd_data=sfoms_quad.suboptimality_data-f_opt

# Retrieve the mean and std of paths
stagd_path_mean=stagd_data.mean(1)
stagd_path_std=stagd_data.std(1)

# Train risk-averse GMM
print("Train risk-averse GMM")
sfoms_quad.train(learning_rate=ra_gmm_a,
           iter_momentum=ra_gmm_b,
           grad_momentum=ra_gmm_g,
           Samples=sample_size,
           stop_criteria=max_iteration,
           seed=1)
ragmm_data=sfoms_quad.suboptimality_data-f_opt
# Retrieve the mean and std of paths
ragmm_path_mean=ragmm_data.mean(1)
ragmm_path_std=ragmm_data.std(1)


# Train risk-averse AGD
print("Train risk-averse AGD")
sfoms_quad.train(learning_rate=ra_agd_a,
           iter_momentum=ra_agd_b,
           grad_momentum=ra_agd_g,
           Samples=sample_size,
           stop_criteria=max_iteration,
           seed=1)
raagd_data=sfoms_quad.suboptimality_data-f_opt

# Retrieve the mean and std of paths
raagd_path_mean=raagd_data.mean(1)
raagd_path_std=raagd_data.std(1)

INFO:root:Training model...
learning rate |  0.010 | Iteration momentum |  0.000 | Gradient Momentum |  0.000
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 0.0088| Norm of gradient:  3.8927
INFO:root:Iteration: 100| Function Value: -0.0010| Norm of gradient:  3.2282
INFO:root:Iteration: 150| Function Value: 0.0162| Norm of gradient:  4.3478
INFO:root:Iteration: 200| Function Value: -0.0074| Norm of gradient:  3.2176
INFO:root:Iteration: 250| Function Value: 0.0335| Norm of gradient:  2.7239
INFO:root:Iteration: 300| Function Value: -0.0194| Norm of gradient:  4.0053


Train standard GD


INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/quadratic_subopt_GD_Var_1.csv
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:Training model...
learning rate |  0.010 | Iteration momentum |  0.614 | Gradient Momentum |  0.614
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 0.0413| Norm of gradient:  4.1619
INFO:root:Iteration: 100| Function Value: 0.0114| Norm of gradient:  3.3765
INFO:root:Iteration: 150| Function Value: 0.0259| Norm of gradient:  4.4290
INFO:root:Iteration: 200| Function Value: 0.0320| Norm of gradient:  3.3255
INFO:root:Iteration: 250| Function Value: 0.0477| Norm of gradient:  2.8073
INFO:root:Iteration: 300| Function Value: -0.0126| Norm of gradient:  4.0490


Standard AGD


INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/quadratic_subopt_AGD_Var_1.csv
INFO:root:Training model...
learning rate |  0.000 | Iteration momentum |  0.959 | Gradient Momentum |  1.000
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 4.3563| Norm of gradient:  18.5225
INFO:root:Iteration: 100| Function Value: 0.1984| Norm of gradient:  5.3306
INFO:root:Iteration: 150| Function Value: 0.0085| Norm of gradient:  2.9904
INFO:root:Iteration: 200| Function Value: -0.0154| Norm of gradient:  3.1370
INFO:root:Iteration: 250| Function Value: -0.0040| Norm of gradient:  2.9565
INFO:root:Iteration: 300| Function Value: 0.0010| Norm of gradient:  4.3514


Risk-averse GMM


INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/quadratic_subopt_TMM_Var_1.csv
INFO:root:Training model...
learning rate |  0.000 | Iteration momentum |  0.959 | Gradient Momentum |  0.959
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 4.5920| Norm of gradient:  19.3166
INFO:root:Iteration: 100| Function Value: 0.2132| Norm of gradient:  5.5011
INFO:root:Iteration: 150| Function Value: 0.0105| Norm of gradient:  3.0050
INFO:root:Iteration: 200| Function Value: -0.0155| Norm of gradient:  3.1524
INFO:root:Iteration: 250| Function Value: -0.0040| Norm of gradient:  2.9672
INFO:root:Iteration: 300| Function Value: 0.0012| Norm of gradient:  4.3523


Risk-averse AGD


INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/quadratic_subopt_AGD_Var_1.csv


In the following figures, we plot the suboptimality, $f(x_k)-f(x_*)$, for the iterations $\{x_k\}$ generated by the algorithms. 

In [43]:
# Suboptimality Figures
subopt_fig=go.Figure()
scale=1


# Plot risk-averse GMM
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean+scale*ragmm_path_std),
                                line=dict(width=0,color="rgba(255, 0, 0, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean-scale*ragmm_path_std)
                                ,line=dict(width=0,color="rgba(255, 0, 0, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean),
                                line=dict(color="rgba(255, 0, 0, 1)",width=5),
                                name="RA-GMM"))

# Plot risk-averse AGD
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean+scale*raagd_path_std),
                                line=dict(width=0,color="rgba(250, 15, 250, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean-scale*raagd_path_std)
                                ,line=dict(width=0,color="rgba(250, 15, 250, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend= False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean),
                                line=dict(color="rgba(250, 15, 250, 1)",width=5),
                                name="RA-AGD"))

# Plot standard AGD
subopt_fig.add_trace(go.Scatter(y= np.log(stagd_path_mean+scale*stagd_path_std),
                                line=dict(width=0,color="rgba(45, 200, 100, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean-scale*stagd_path_std)
                                ,line=dict(width=0,color="rgba(45, 200, 100, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean),
                                line=dict(color="rgba(45, 200, 100, 1)",width=5),
                                name="AGD"))

# Plot standard SGD
subopt_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean+scale*stgd_path_st),
                                line=dict(width=0,color="rgba(0, 80, 255, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y= np.log(stgd_path_mean-scale*stgd_path_st)
                                ,line=dict(width=0,color="rgba(0, 80, 255, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean),
                                line=dict(color="rgba(0, 80, 255, 1)",width=5),
                                name="GD"))
subopt_fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.95,
    xanchor="right",
    x=0.8
),
legend_font_size=25,
yaxis_title=r"$\log(f(x_k)-f(x_*))$",
xaxis_title="Iterations",
font=dict(size=20),
width=1000,
height=800)
# Toggle between log scale on the axes.
# subopt_fig.update_yaxes(type="log")
# subopt_fig.update_xaxes(type="log")
subopt_fig.show()



We also compare the the convergence of the algorithms in terms of empirical finite-horizon entropic risk measure, $\tilde{r}_{k,\sigma^2}(\theta)$. We accomplished this by sampling $\{x_k^{(i)}\}_{i=1,..,50}$ at each iteration and computing 
$$
\tilde{r}_{k,\sigma^2}(\theta):= \frac{2\sigma^2}{\theta}\log \frac{1}{50} \sum_{i=1}^{50} e^{\frac{\theta}{2\sigma^2}(f(x_k^{(i)})-f(x_*))},
$$

In [44]:
# The comparison of convergence with respect to empirical risk measure
theta=5

# Empirical risk measure of ST-GD
diff_gd=stgd_data.values
emp_risk_meas_stgd=sfoms_quad.emp_risk_meas(diff_gd,theta)

# Empirical risk measure of ST-AGD
diff_stagd=stagd_data.values
emp_risk_meas_stagd=sfoms_quad.emp_risk_meas(diff_stagd,theta)

# Empirical risk measure of RA-AGD
diff_raagd=raagd_data.values
emp_risk_meas_raagd=sfoms_quad.emp_risk_meas(diff_raagd,theta)

# Emprical risk measure of RA-GMM
diff_ragmm=ragmm_data.values
emp_risk_meas_ragmm=sfoms_quad.emp_risk_meas(diff_ragmm,theta)


risk_meas_fig=go.Figure()
# Plot emp. risk measure of standard SGD 
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_stgd),
                                line=dict(color="rgba(0, 80, 255, 1)",width=5),
                                # name="ST-GD (risk meas)",
                                name="GD"
                                ))
risk_meas_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean),
                                line=dict(color="rgba(0, 80, 255, 1)",dash="dash"),
                                # name="ST-GD (mean)"
                                showlegend=False
                                ))
# Plot emp. risk measure of standard AGD
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_stagd),
                                line=dict(color="rgba(45, 200, 100, 1)",width=5),
                                # name="ST-AGD (risk meas)",
                                name="AGD"
                                ))
risk_meas_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean),
                                line=dict(color="rgba(45, 200, 100, 1)",dash='dash'),
                                # name="ST-AGD (mean)"
                                showlegend=False
                                ))

# Plot risk-averse AGD
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_raagd),
                                line=dict(color="rgba(250, 15, 250, 1)",width=5),
                                # name="raAGD-risk meas"
                                name="RA-AGD"
                                )
                                )
risk_meas_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean),
                                line=dict(color="rgba(250, 15, 250, 1)",dash="dash"),
                                name="RA-AGD (mean)",
                                showlegend=False
                                ))


# Plot risk-averse GMM
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_ragmm),
                                line=dict(color="rgba(255, 0, 0, 1)",width=5),
                                name="RA-GMM"
                                ))
risk_meas_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean),
                                line=dict(color="rgba(255, 0, 0, 1)",dash='dash'),
                                name="RA-GMM (mean)",
                                showlegend=False)
)

risk_meas_fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.95,
    xanchor="right",
    x=0.85
),
legend_font_size=25,
title="Convergence with respect to empirical entropic risk measure.",
yaxis_title=r"$r_{k,\sigma^2}(\theta)$",
xaxis_title="Iterations",
font=dict(size=20),
width=1000,
height=800)
# risk_meas_fig.update_yaxes(type="log")
risk_meas_fig.show()


Lastly, we look at the cumulative distribution functions, $\mathbb{P}\{f(x_k)-f(x_*)\geq t\}$, of the algorithms at the last iterates ($k=300)$.

In [9]:
# The histogram plots
# The distribution of standard GD
stgd_dist=stgd_data.values[-1]

# The distribution of standard AGD
stagd_dist=stagd_data.values[-1]

# The distribution of risk-averse GMM
ragmm_dist=ragmm_data.values[-1]

# The distribution of risk-averse AGD
raagd_dist=raagd_data.values[-1]

# Find the maximum value
bound_max=max(stgd_dist.max(),stagd_dist.max(),ragmm_dist.max(),
             raagd_dist.max())

# The function to create the histogram
def create_hist(dist, bound_max):
    axis=np.linspace(0,bound_max,200)
    hist=[]
    for a in axis:
        hist+=[np.count_nonzero(dist<=a)]
    return hist/np.max(hist)

# Boundary smoothing
def smoothTriangle(data, degree):
    triangle=np.concatenate((np.arange(degree + 1), np.arange(degree)[::-1])) # up then down
    smoothed=[data[0]]
    for i in range(degree+1, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(np.sum(point)/np.sum(triangle))
    # Handle boundaries
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed

axis=np.linspace(0,bound_max,200)

## Histograms
dist_fig=go.Figure()
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(stgd_dist,1.05*bound_max),10),
                              line=dict(color="rgba(0, 80, 255, 1)",width=5),
                              name="GD"))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(stagd_dist,1.05*bound_max),12), 
                              name="AGD",
                              line=dict(color="rgba(45, 200, 100, 1)",width=5)))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(raagd_dist,1.05*bound_max),8), 
                              name="RA-AGD",
                              line=dict(color="rgba(250, 15, 250, 1)",width=5)))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(ragmm_dist,1.05*bound_max),3),
                              name="RA-GMM",
                              line=dict(color="rgba(255, 0, 0, 1)",width=5)))
dist_fig.update_layout(width=800,height=600,
                       legend=dict(orientation="v",
                                   xanchor="right",
                                   x=0.99,
                                   yanchor="bottom",
                                   y=0.01,
                                   font=dict(size=40)),
                       title="Histogram of limiting distribution",
                       xaxis_title=r"$f(x_{300})-f(x_*)$",
                       font=dict(size=20),
                       legend_font_size=25
                       )
dist_fig.show()



## 2. Logistic Regression on Synthetic Data
We consider the regularized binary logistic regression problem for which the objective function is given as
$$
f(x)=\frac{1}{N}\sum_{i=1}^{N} \log(1+\exp\{-y_i (X_i^\top x) \})+\frac{\lambda_{{reg}}}{2}\Vert x\Vert^2,
$$
where $X_i\in\mathbb{R}^{100}$ is the feature vector and $y_i\in \{-1,1\}$ is the label of $i$-th sample. We set the regularization parameter $\lambda_{\tiny\mbox{reg}}=1$, the number of features $d=100$, and the sample size $N=1000$. 

We generated our synthetic data by sampling a random vector $x_0\in\mathbb{R}^{100}$ from a standard Gaussian distribution $\mathcal{N}(0,I)$. We also generated data from Gaussian distribution, i.e. $X\sim \mathcal{N}(0,\sigma_X I)$. After that we created the labels by $y_i=1 $ if $X_i^\top x_0\geq 0$ and $-1$ otherwise.  

We define both deterministic model, (det_logit) and stochastic model, (sfoms_logit). We generated the synthetic data:

In [48]:
# Problem Setup and Data Generation
regularizer =  1 # \lambda_{reg}
noise_std = 1    # \sigma

# Define the models
det_logit=entropic_ra_sFoms(optim_problem="synthetic_logit"
                ,data_std=5
                ,data_size=1000
                ,num_of_feat=100
                ,reg_param=regularizer 
                ,noise_std=0
                ,init_x0="fixed"
                )

sfoms_logit=entropic_ra_sFoms(optim_problem="synthetic_logit"
                  ,data_std=5
                  ,data_size=1000
                  ,num_of_feat=100
                  ,reg_param=regularizer
                  ,noise_std=noise_variance
                  ,init_x0="fixed"
                  )

# Compute the L and mu based on the data.
L, mu = det_logit.L_and_mu_logistic_reg()
print(f"The problem constants\n\
L:{L:.4f}, mu: {mu: .4f}.\n")
# The synthetic data 
print(f"The summary of data:\n\
Number of Samples: {det_logit.X.shape[0]},\n\
Number of Features: {det_logit.X.shape[1]},\n\
Data mean:\n {det_logit.X.mean(0)},\n\
Data variance:\n {det_logit.X.var(0)},")

The problem constants
L:44.1307, mu:  1.0000.

The summary of data:
Number of Samples: 1000,
Number of Features: 100,
Data mean:
 [ 0.20216875 -0.15333205  0.08138669 -0.05879052  0.00477513 -0.12301528
  0.17951487 -0.21033463  0.0723986   0.14914805 -0.2014252   0.04654614
 -0.03377486 -0.153377   -0.05925103  0.3353758   0.17514244 -0.02589386
 -0.16538578  0.04002559  0.11131784  0.11602741 -0.22351584  0.34232515
  0.27602622 -0.18998717 -0.08465082  0.00678498  0.12066452 -0.11897304
  0.46693817  0.37558702  0.23397191 -0.22782938  0.23725934 -0.07200616
 -0.13155568 -0.33425552  0.15068372  0.06552859  0.2073983   0.19858058
  0.01150587  0.24683128  0.33338344  0.04601927 -0.30333075  0.05173474
  0.0548528  -0.08445488  0.1740978  -0.06402054  0.19429387 -0.2554275
  0.04877542 -0.08824936 -0.025205   -0.09987336 -0.19099496  0.17091723
  0.13232927 -0.18968837 -0.07824177  0.03965746 -0.06917621  0.12324177
  0.06302888 -0.11473883  0.2407992  -0.31409284 -0.17846034  0.0510

We train deterministic model with standard GD to compute $f(x_*)$, so that we can plot $f(x_k)-f(x_*)$ and calculate entropic risk measure and the CDF of limiting distributions.

In [49]:
# Train deterministic GD to obtain the solution
det_logit.train(learning_rate=1/L
                  ,stop_criteria=None
                  ,Samples=1,
                  seed=1)

det_path=det_logit.suboptimality_data.values.T[0]

# Plot the function values per iteration during training
det_quad_fig=go.Figure()
det_quad_fig.add_trace(go.Scatter(y=det_path))
det_quad_fig.update_layout(title=r"$f(x_k)\text{ for deterministic GD}$",
                           yaxis_title=r"$\log(f(x_k))$")
det_quad_fig.update_yaxes(type="log")
det_quad_fig.show()

# Retrieve f(x_*) from deterministic GD
f_opt=det_logit.suboptimality_data.values[-1][0]

INFO:root:Training model...
learning rate |  0.023 | Iteration momentum |  0.000 | Gradient Momentum |  0.000
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 4.5522| Norm of gradient:  3.4569
INFO:root:Iteration: 100| Function Value: 0.4095| Norm of gradient:  0.5444
INFO:root:Iteration: 150| Function Value: 0.3357| Norm of gradient:  0.0474
INFO:root:Iteration: 200| Function Value: 0.3352| Norm of gradient:  0.0036
INFO:root:Iteration: 250| Function Value: 0.3352| Norm of gradient:  0.0003
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/synthetic_logit_subopt_GD_Var_0.csv


We find the parameters of risk-averse GMM and risk-averse AGD based on the following problem provided in [[Can and Gurbuzbalaban, 2022]](https://arxiv.org/abs/2204.11292):
$$
(\vartheta_\star,\psi_\star)=\underset{(\vartheta,\psi)\in \mathcal{S}_c \cup \mathcal{S}_0}{\text{argmin}}\; \bar{E}_{1-\zeta}(\vartheta,\psi)\\
    \text{s.t.}\quad \frac{\rho_{\vartheta,\psi}^2}{\rho_*^2} \leq (1+\varepsilon).
$$
The parameter $\varepsilon$ is a trade-off parameter as before, it captures what percent of rate we want to give up from the fastest convergence rate $\rho_*^2=1-\sqrt{\mu/L}$ to ensure better entropic risk bounds.

In [55]:
#@title The parameters of risk averse TMM and risk averse AGD
trade_off= 0.05 #@param {type: "raw"}
confidence_level= 0.95 #@param {type: "raw"}
dimension=det_logit.num_of_feat

# Retrieve the parameters of risk averse GMM
ra_gmm_a, ra_gmm_b, ra_gmm_g= sfoms_logit.raTMM_str_cnv_params(L,mu,trade_off,dimension,confidence=confidence_level)
print(f"The RA-GMM parameters:\n\
learn_rate:{ra_gmm_a: .4f}, iter_momentum:{ra_gmm_b: .5f}, grad_momentum:{ra_gmm_g: .5f} \n")

# Retrieve the parameres of risk averse AGD
ra_agd_a, ra_agd_b, ra_agd_g= sfoms_logit.raAGD_str_cnv_params(L,mu,trade_off,dimension,confidence=confidence_level)
print(f"The RA-AGD parameters:\n\
learn_rate:{ra_agd_a: .4f}, iter_momentum:{ra_agd_b: .4f}, grad_momentum:{ra_agd_g: .4f}")

The RA-GMM parameters:
learn_rate: 0.0003, iter_momentum: 0.96590, grad_momentum: 0.25113 

The RA-AGD parameters:
learn_rate: 0.0004, iter_momentum: 0.9586, grad_momentum: 0.9586


We train the logistic regression model on synthetic using standard and risk-averse algorithms.

In [51]:
# Train the models on synthethic data
max_iteration=600
sample_size=50
# Train standard GD
print("Train standard GD\n") 
stgd_a=1/L
sfoms_logit.train(learning_rate=stgd_a,
                 Samples=sample_size,
                 stop_criteria=max_iteration, 
                 seed=1)
stgd_data=sfoms_logit.suboptimality_data-f_opt
# Retrieve the mean and std of paths
stgd_path_mean= stgd_data.mean(1)
stgd_path_st= stgd_data.std(1)

# Train standard AGD
print("Train standard AGD")
print("\n")
stagd_a=1/L
stagd_b= (np.sqrt(L)-np.sqrt(mu))/(np.sqrt(L)+np.sqrt(mu))
sfoms_logit.train(learning_rate=stagd_a,
                 iter_momentum=stagd_b,
                 grad_momentum=stagd_b,
                 Samples=sample_size,
                 stop_criteria=max_iteration,
                 seed=1)

stagd_data=sfoms_logit.suboptimality_data-f_opt
# Retrieve the mean and std of paths
stagd_path_mean=stagd_data.mean(1)
stagd_path_std=stagd_data.std(1)

# Train risk-averse GMM
print("Train risk-averse GMM\n")
sfoms_logit.train(learning_rate=ra_gmm_a,
           iter_momentum=ra_gmm_b,
           grad_momentum=ra_gmm_g,
           Samples=sample_size,
           stop_criteria=max_iteration,
           seed=1)
ragmm_data=sfoms_logit.suboptimality_data-f_opt
# Retrieve the mean and std of paths
ragmm_path_mean=ragmm_data.mean(1)
ragmm_path_std=ragmm_data.std(1)


# Train risk-averse AGD
print("Train risk-averse TMM\n")
sfoms_logit.train(learning_rate=ra_agd_a,
           iter_momentum=ra_agd_b,
           grad_momentum=ra_agd_g,
           Samples=sample_size,
           stop_criteria=max_iteration,
           seed=1)
raagd_data=sfoms_logit.suboptimality_data-f_opt
# Retrieve the mean and std of paths
raagd_path_mean=raagd_data.mean(1)
raagd_path_std=raagd_data.std(1)

INFO:root:Training model...
learning rate |  0.023 | Iteration momentum |  0.000 | Gradient Momentum |  0.000
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 4.8998| Norm of gradient:  9.1085
INFO:root:Iteration: 100| Function Value: 1.1899| Norm of gradient:  11.8195
INFO:root:Iteration: 150| Function Value: 1.0832| Norm of gradient:  11.0394
INFO:root:Iteration: 200| Function Value: 1.0519| Norm of gradient:  10.3110
INFO:root:Iteration: 250| Function Value: 0.8829| Norm of gradient:  10.9687


Train standard GD



INFO:root:Iteration: 300| Function Value: 0.9973| Norm of gradient:  10.3574
INFO:root:Iteration: 350| Function Value: 1.0575| Norm of gradient:  9.3708
INFO:root:Iteration: 400| Function Value: 1.1353| Norm of gradient:  11.3147
INFO:root:Iteration: 450| Function Value: 1.1472| Norm of gradient:  9.6542
INFO:root:Iteration: 500| Function Value: 1.1914| Norm of gradient:  8.6135
INFO:root:Iteration: 550| Function Value: 1.0626| Norm of gradient:  10.7386
INFO:root:Iteration: 600| Function Value: 1.0274| Norm of gradient:  10.5319
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/synthetic_logit_subopt_GD_Var_1.csv
INFO:root:Training model...
learning rate |  0.023 | Iteration momentum |  0.738 | Gradient Momentum |  0.738
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 2.3921| Norm of gradient:  8.9885
INFO:root:Iteration: 100|

Train standard AGD




INFO:root:Iteration: 300| Function Value: 2.6713| Norm of gradient:  10.4915
INFO:root:Iteration: 350| Function Value: 3.1230| Norm of gradient:  9.8938
INFO:root:Iteration: 400| Function Value: 2.9568| Norm of gradient:  11.3940
INFO:root:Iteration: 450| Function Value: 2.8944| Norm of gradient:  9.8210
INFO:root:Iteration: 500| Function Value: 3.3263| Norm of gradient:  8.9254
INFO:root:Iteration: 550| Function Value: 2.3923| Norm of gradient:  11.0109
INFO:root:Iteration: 600| Function Value: 2.7022| Norm of gradient:  10.8211
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/synthetic_logit_subopt_AGD_Var_1.csv
INFO:root:Training model...
learning rate |  0.000 | Iteration momentum |  0.966 | Gradient Momentum |  0.251
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 40.3485| Norm of gradient:  12.4529
INFO:root:Iteration: 1

Train risk-averse GMM



INFO:root:Iteration: 300| Function Value: 0.6297| Norm of gradient:  10.2863
INFO:root:Iteration: 350| Function Value: 0.5603| Norm of gradient:  8.8075
INFO:root:Iteration: 400| Function Value: 0.5278| Norm of gradient:  11.1474
INFO:root:Iteration: 450| Function Value: 0.5816| Norm of gradient:  9.6199
INFO:root:Iteration: 500| Function Value: 0.6560| Norm of gradient:  8.4037
INFO:root:Iteration: 550| Function Value: 0.6007| Norm of gradient:  10.5772
INFO:root:Iteration: 600| Function Value: 0.5622| Norm of gradient:  10.3983
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/synthetic_logit_subopt_TMM_Var_1.csv
INFO:root:Training model...
learning rate |  0.000 | Iteration momentum |  0.959 | Gradient Momentum |  0.959
INFO:root:Iterations of 1st sample:
INFO:root:Iteration: 50| Function Value: 32.4016| Norm of gradient:  11.8107
INFO:root:Iteration: 1

Train risk-averse TMM



INFO:root:Iteration: 300| Function Value: 0.6263| Norm of gradient:  10.3891
INFO:root:Iteration: 350| Function Value: 0.6058| Norm of gradient:  8.8683
INFO:root:Iteration: 400| Function Value: 0.5786| Norm of gradient:  11.1465
INFO:root:Iteration: 450| Function Value: 0.6641| Norm of gradient:  9.6055
INFO:root:Iteration: 500| Function Value: 0.7079| Norm of gradient:  8.4339
INFO:root:Iteration: 550| Function Value: 0.6490| Norm of gradient:  10.5337
INFO:root:Iteration: 600| Function Value: 0.6100| Norm of gradient:  10.3821
INFO:root:Saved the suboptimality results into the path: 
/Users/bugracan/Library/Mobile Documents/com~apple~CloudDocs/Research/Code Repo/Risk-Averse GMM/risk-averse GMM experiments/synthetic_logit_subopt_AGD_Var_1.csv


We plot the suboptimality, $f(x_k)-f(x_*)$, at the iterates $\{x_k\}$ generated by the algorithms.

In [52]:
#@title The suboptimality figures 
subopt_fig=go.Figure()
scale=1

# Plot risk-averse GMM
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean+scale*ragmm_path_std),
                                line=dict(width=0,color="rgba(255, 0, 0, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean-scale*ragmm_path_std)
                                ,line=dict(width=0,color="rgba(255, 0, 0, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean),
                                line=dict(color="rgba(255, 0, 0, 1)", width=5),
                                name="RA-GMM"))

# Plot risk-averse AGD
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean+scale*raagd_path_std),
                                line=dict(width=0,color="rgba(250, 15, 250, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean-scale*raagd_path_std)
                                ,line=dict(width=0,color="rgba(250, 15, 250, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend= False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean),
                                line=dict(color="rgba(250, 15, 250, 1)",width=5),
                                name="RA-AGD"))

# Plot standard AGD
subopt_fig.add_trace(go.Scatter(y= np.log(stagd_path_mean+scale*stagd_path_std),
                                line=dict(width=0,color="rgba(45, 200, 100, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean-scale*stagd_path_std)
                                ,line=dict(width=0,color="rgba(45, 200, 100, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean),
                                line=dict(color="rgba(45, 200, 100, 1)",width=5),
                                name="AGD"))


# Plot standard SGD
subopt_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean+scale*stgd_path_st),
                                line=dict(width=0,color="rgba(0, 80, 255, 0.5)"),
                                showlegend=False))
subopt_fig.add_trace(go.Scatter(y= np.log(stgd_path_mean-scale*stgd_path_st)
                                ,line=dict(width=0,color="rgba(0, 80, 255, 0.5)")
                                ,fill="tonexty"
                                ,name=r"$\text{mean} \pm "+str(scale)+ r"\text{std}$"
                                ,showlegend=False
                                ))
subopt_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean),
                                line=dict(color="rgba(0, 80, 255, 1)",width=5),
                                name="GD"))



subopt_fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.95,
    xanchor="right",
    x=0.85
),
legend_font_size=25,
yaxis_title=r"$\log(f(x_k)-f(x_*))$",
xaxis_title="Iterations",
font=dict(size=20),
width=1000,
height=800)
# subopt_fig.update_yaxes(type="log")
# subopt_fig.update_xaxes(type="log")
subopt_fig.show()

The convergence with respect to emprical entropic risk measure, $\tilde{r}_{k,\sigma^2}$. 

In [57]:
#@title Empirical $r_{k,\sigma^2}(\theta)$ comparison
theta=5 #@param {type: "raw"}

# Empirical risk measure of ST-GD
diff_gd=stgd_data.values
emp_risk_meas_stgd=sfoms_logit.emp_risk_meas(diff_gd,theta)

# Empirical risk measure of ST-AGD
diff_stagd=stagd_data.values
emp_risk_meas_stagd=sfoms_logit.emp_risk_meas(diff_stagd,theta)

# Empirical risk measure of RA-AGD
diff_raagd=raagd_data.values
emp_risk_meas_raagd=sfoms_logit.emp_risk_meas(diff_raagd,theta)

# Emprical risk measure of RA-GMM
diff_ragmm=ragmm_data.values
emp_risk_meas_ragmm=sfoms_logit.emp_risk_meas(diff_ragmm,theta)


risk_meas_fig=go.Figure()
# Plot emp. risk measure of standard SGD 
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_stgd),
                                line=dict(color="rgba(0, 80, 255, 1)",width=3),
                                # name="stGD-risk meas",
                                name="GD"
                                ))
# risk_meas_fig.add_trace(go.Scatter(y=np.log(stgd_path_mean),
#                                 line=dict(color="rgba(0, 80, 255, 1)",dash="dash",width=3),
#                                 # name="stGD-mean"
#                                 showlegend=False
#                                 ))
# Plot emp. risk measure of standard AGD
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_stagd),
                                line=dict(color="rgba(45, 200, 100, 1)",width=3),
                                # name="stAGD-risk meas",
                                name="AGD"
                                ))
# risk_meas_fig.add_trace(go.Scatter(y=np.log(stagd_path_mean),
#                                 line=dict(color="rgba(45, 200, 100, 1)",dash='dash',width=3),
#                                 # name="stAGD-mean"
#                                 showlegend=False
#                                 ))

# Plot risk-averse AGD
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_raagd),
                                line=dict(color="rgba(250, 15, 250, 1)",width=3),
                                # name="raAGD-risk meas"
                                name="RA-AGD")
                                )
# risk_meas_fig.add_trace(go.Scatter(y=np.log(raagd_path_mean),
#                                 line=dict(color="rgba(250, 15, 250, 1)",dash="dash",width=3),
#                                 name="RA-AGD-mean",
#                                 showlegend=False
#                                 ))


# Plot risk-averse GMM
risk_meas_fig.add_trace(go.Scatter(y=np.log(emp_risk_meas_ragmm),
                                line=dict(color="rgba(255, 0, 0, 1)",width=3),
                                # name="raTMM-risk meas",
                                name="RA-GMM"
                                ))
# risk_meas_fig.add_trace(go.Scatter(y=np.log(ragmm_path_mean),
#                                 line=dict(color="rgba(255, 0, 0, 1)",dash='dash',width=3),
#                                 name=r"$\textbf{raTMM-mean}$",
#                                 showlegend=False))



risk_meas_fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.95,
    xanchor="right",
    x=0.85
),
legend_font_size=25,
title="Empirical entropic risk measure per iteration",
yaxis_title=r"$r_{k,\sigma^2}(\theta)$",
xaxis_title="Iterations",
font=dict(size=20),
width=1000,
height=800)
# risk_meas_fig.update_yaxes(type="log")
risk_meas_fig.show()

Lastly, we plotted the cumulative distribution functions of the empirical distributions of the last iterates generated by the algorithms.

In [54]:
#@title The histogram plots
# The distribution of standard SGD
stgd_dist=stgd_data.values[-1]

# The distribution of standard AGD
stagd_dist=stagd_data.values[-1]

# The distribution of risk-averse GMM
ragmm_dist=ragmm_data.values[-1]

# The distribution of risk-averse AGD
raagd_dist=raagd_data.values[-1]

# Find the maximum value
bound_max=max(stgd_dist.max(),stagd_dist.max(),ragmm_dist.max(),
             raagd_dist.max())

# The function to create the histogram
def create_hist(dist, bound_max):
    axis=np.linspace(0,bound_max,200)
    hist=[]
    for a in axis:
        hist+=[np.count_nonzero(dist<=a)]
    return hist/np.max(hist)

# Boundary smoothing
def smoothTriangle(data, degree):
    triangle=np.concatenate((np.arange(degree + 1), np.arange(degree)[::-1])) # up then down
    smoothed=[data[0]]
    for i in range(degree+1, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(np.sum(point)/np.sum(triangle))
    # Handle boundaries
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    smoothed.append(data[-1])
    return smoothed

axis=np.linspace(0,1.05*bound_max,200)

## Histograms
dist_fig=go.Figure()
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(stgd_dist,1.05*bound_max),7),
                              line=dict(color="rgba(0, 80, 255, 1)",width=5),
                              name="GD"))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(stagd_dist,1.05*bound_max),9), 
                              name="AGD",
                              line=dict(color="rgba(45, 200, 100, 1)",width=5)))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(raagd_dist,1.05*bound_max),3), 
                              name="RA-AGD",
                              line=dict(color="rgba(250, 15, 250, 1)", width=5)))
dist_fig.add_trace(go.Scatter(x=axis,y=smoothTriangle(create_hist(ragmm_dist,1.05*bound_max),3), 
                              name="RA-GMM",
                              line=dict(color="rgba(255, 0, 0, 1)",width=5)))
dist_fig.update_layout(width=800,height=600,
                       legend=dict(orientation="v",
                                   xanchor="right",
                                   x=0.99,
                                   yanchor="bottom",
                                   y=0.01),
                       legend_font_size=25,
                       title="Histogram of limiting distribution",
                       xaxis_title=r"$f(x_{600})-f(x_*)$",
                       font=dict(size=20)
                       )

dist_fig.show()